# Headline

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Phasellus lobortis scelerisque auctor. Maecenas eu ullamcorper eros. Ut aliquet at quam nec fringilla. Nunc sem dui, rhoncus sed dignissim a, dapibus ut mi. Nulla nisi nunc, scelerisque faucibus libero id, scelerisque rutrum nisl. Sed non aliquet lorem. Interdum et malesuada fames ac ante ipsum primis in faucibus. Maecenas magna nisl, pulvinar in nisi in, accumsan ornare lectus. Etiam semper mi tortor, sed consectetur sem iaculis a. Aenean laoreet eros at pulvinar fermentum.

In [50]:
from langchain import hub
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain_qdrant import QdrantVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from tqdm import tqdm


## Load documents

In [10]:
loader = PyPDFDirectoryLoader("PDFs/")
documents = loader.load()
print(len(documents))

107


Each document corresponds to one page in a PDF file. Let us explore the content of the first document

In [41]:
print(f"{documents[0].page_content[:500]}\n")


Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE FISCAL YEAR ENDED MAY 31, 2023
OR
☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE TRANSITION PERIOD FROM                         TO                         .
Commission File No. 1-10635
NIKE, Inc.
(Exact name of Registrant as specified in it



and the corresponding metadata

In [39]:
print(documents[0].metadata)


{'source': 'PDFs/nke-10k-2023.pdf', 'page': 0, 'page_label': '1'}


More advanced method of extracting text and multimodal

## Text splitter

Split each page into smaller chunks. 

add_start_index=True ensures meta data is preserved.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
# https://python.langchain.com/docs/how_to/split_by_token/

# from langchain_text_splitters import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     model_name="gpt-4",
#     chunk_size=1000,
#     chunk_overlap=200,
#     add_start_index=True
# )

all_splits = text_splitter.split_documents(documents)

len(all_splits)

135

In [18]:
all_splits[12]

Document(metadata={'source': 'PDFs/nke-10k-2023.pdf', 'page': 3, 'page_label': '4', 'start_index': 2264}, page_content='We also sell sports apparel, which features the same trademarks and are sold predominantly through the same marketing and distribution channels as athletic footwear.\nOur sports apparel, similar to our athletic footwear products, is designed primarily for athletic use, although many of the products are worn for casual or leisure purposes,\nand demonstrates our commitment to innovation and high-quality construction. Our Men\'s and Women\'s apparel products currently lead in apparel sales and we expect\nthem to continue to do so. We often market footwear, apparel and accessories in "collections" of similar use or by category. We also market apparel with licensed college\nand professional team and league logos.\nWe sell a line of performance equipment and accessories under the NIKE Brand name, including bags, socks, sport balls, eyewear, timepieces, digital devices, bats

## Embeddings

In [53]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [15]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.04747234284877777, 0.021675830706954002, -0.009018060751259327, 0.0053567527793347836, 0.025557713583111763, -0.010230296291410923, -0.008413949981331825, 0.039303891360759735, 0.021570555865764618, -0.024095403030514717]


## How does embed_query work - dive deep

In [33]:
client = QdrantClient("http://localhost:6333")

collection_exists = client.collection_exists(collection_name="test2")

if not collection_exists:
    client.create_collection(
        collection_name="test2",
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )

vector_store = QdrantVectorStore(
    client=client,
    collection_name="test2",
    embedding=embeddings,
)


In [57]:
ids = vector_store.add_documents(documents=all_splits)

In [35]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'source': 'PDFs/nke-10k-2023.pdf', 'page': 26, 'pag

In [28]:
llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0)

In [43]:
# See full prompt at https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vector_store.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("Does Nike have an office in China?")

/home/carstenj/dev/qdrant/.venv/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'<think>\nOkay, so I need to figure out if Nike has an office in China based on the provided context. Let me read through the information carefully.\n\nFirst, looking at the table of contents, there\'s a section titled "PROPERTIES." Under that, it lists several properties owned or leased by Nike. The first property mentioned is the NIKE World Campus in Beaverton, Oregon, USA. It says they occupy approximately 11,400 employees and manage various functions like management, research, design, etc.\n\nThen there\'s a smaller administrative facility in Hilversum, Netherlands, which serves as headquarters for Europe, Middle East & Africa. Next, under "Operations," it mentions an office complex in Shanghai, China, serving Greater China geography with employees focused on implementing their operations.\n\nAdditionally, the context talks about distribution centers in the United States: five near Memphis, two owned and three leased; one in Indianapolis and one in Dayton, both leased by logistics 